# Solution to Exercise 09 - Reasoning with SWRL

In today's exercise we will be using the Python library *owlready2* again to create and reason with rules written in the SWRL language.
A quick overview over the usage of SWRL in *owlready2* can be found in the [documentation](https://owlready2.readthedocs.io/en/latest/rule.html) and a reference for SWRL, including the different build-ins, exists at the [W3 Compendium](https://www.w3.org/submissions/SWRL/).

As an ontology, we will be using the domain of Pokémon, similar to last week's exercise.
However, since we needed to make some adaptations to the underlying ontology, please use the provided one found in the Moodle course (PokemonOntology.owl).

## Setup

In [ ]:
# First we install the library
!pip install owlready2

In [ ]:
# Next we import the necessary libraries
from owlready2 import *

In [ ]:
# Continue with loading the ontology and defining the namespace
pkmn_onto = get_ontology("./PokemonOntology.owl").load()

# Check the available classes
print(list(Thing.subclasses()))

## Solution 01 - Pokémon Generations

For the first exercise, we want to diversify the simple *Pokémon* class using complex classes that use class axioms to automatically assing Pokémon to the generation they belong to ($\in [1, 4]$).
This classification is based on the national dex number of each Pokémon.
Below you can find the concrete numbers, please create four new classes that follow the general class axioms discussed in the last lecture.

Generation 1: $\in [001, 151]$ 

Generation 2: $\in [152, 251]$ 

Generation 3: $\in [252, 386]$

Generation 4: $\in [387, 493]$ 

In [ ]:
with pkmn_onto:
    class Gen1Pokemon(pkmn_onto.Pokemon):
        equivalent_to = [pkmn_onto.Pokemon & (pkmn_onto.nationalDexNumber <= 151)]
        
    class Gen2Pokemon(pkmn_onto.Pokemon):
        equivalent_to = [pkmn_onto.Pokemon & (pkmn_onto.nationalDexNumber >= 152) & (pkmn_onto.nationalDexNumber <= 251)]

    class Gen3Pokemon(pkmn_onto.Pokemon):
        equivalent_to = [pkmn_onto.Pokemon & (pkmn_onto.nationalDexNumber >= 252) & (pkmn_onto.nationalDexNumber <= 386)]

    class Gen4Pokemon(pkmn_onto.Pokemon):
        equivalent_to = [pkmn_onto.Pokemon & (pkmn_onto.nationalDexNumber >= 387) & (pkmn_onto.nationalDexNumber <= 493)]

    # Run the reasoner
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

In [ ]:
# Save the ontology to check 
pkmn_onto.save(file = "PokemonOntologyEx01.owl", format = "rdfxml")

## Solution 02 - Pokémon Stats

For the second exercise, we will be extending the ontology to cover the different stats of each Pokemon. There are six in total: HP, Attack, Defense, Special Attack, Special Defense and Speed. With these stats created, we can add a rule that automatically calculates the BST (Base Stat Total), which is the sum of all six stats. Based on this BST, we can decide whether a Pokemon is powerful (BST >= 550) or not (BST < 550). Please adapt the ontology accordingly and add the SWRL rules.

To verify your result, you can arbitrary stats to some Pokemon. If you want to be realistic, here are some Pokemon and their stats (in the order described above):

Abra = [25, 20, 15, 105, 55, 90]

Tyranitar = [100, 134, 110, 95, 100, 61]

Haunter = [45, 50, 45, 115, 55, 95]

In [ ]:
# Add the data properties
with pkmn_onto:
    class hp(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]
        
    class attack(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class defense(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class specialAttack(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class specialDefense(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class speed(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class bst(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [int]

    class isPowerful(DataProperty):
        domain = [pkmn_onto.Pokemon]
        range = [bool]

In [ ]:
# Add exemplary individuals
with pkmn_onto:
    # Abra = [25, 20, 15, 105, 55, 90]
    abra = pkmn_onto.Abra
    abra.hp.append(25)
    abra.attack.append(20)
    abra.defense.append(15)
    abra.specialAttack.append(105)
    abra.specialDefense.append(55)
    abra.speed.append(90)
    # Tyranitar = [100, 134, 110, 95, 100, 61]
    tyranitar = pkmn_onto.Tyranitar
    tyranitar.hp.append(100)
    tyranitar.attack.append(134)
    tyranitar.defense.append(110)
    tyranitar.specialAttack.append(95)
    tyranitar.specialDefense.append(100)
    tyranitar.speed.append(61)
    # Haunter = [45, 50, 45, 115, 55, 95]
    haunter = pkmn_onto.Haunter
    haunter.hp.append(45)
    haunter.attack.append(50)
    haunter.defense.append(45)
    haunter.specialAttack.append(115)
    haunter.specialDefense.append(55)
    haunter.speed.append(95)

In [ ]:
# Add the rules
with pkmn_onto:
    # Rule 1 for calculating the BST
    bst_rule = Imp()
    bst_rule.set_as_rule("""Pokemon(?p), hp(?p, ?h), attack(?p, ?a), defense(?p, ?d), specialAttack(?p, ?sa), specialDefense(?p, ?sd), speed(?p, ?s), add(?bst, ?h, ?a, ?d, ?sa, ?sd, ?s) -> bst(?p, ?bst)""")

    # Rule 2 for deciding whether a Pokemon is powerful
    power_rule = Imp()
    power_rule.set_as_rule("""Pokemon(?p), bst(?p, ?bst), greaterThanOrEqual(?bst, 550) -> isPowerful(?p, true)""")
    
    # Run the reasoner
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

In [ ]:
# Save the ontology to check 
pkmn_onto.save(file = "PokemonOntologyEx02.owl", format = "rdfxml")